In [244]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict


In [245]:
loginInfo = json.load(open("login.json"))

sf = Salesforce(
    username=loginInfo[0]["username"],
    password=loginInfo[0]["password"],
    security_token=loginInfo[0]["security_token"],
)
paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [3]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)


2023-09-05
05092023
VentaCARE_ASSISTANCE_05092023.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_05092023.xlsx


In [4]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)


Coneccion a carpeta sft y rescate de archivo


In [5]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)


In [6]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()


Inicio Manipulacion de Excel descargado


In [7]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(2436, 17)

In [8]:
archivoBCI.dtypes

POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [9]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)


,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,25353688,8,MASCULINO,HIJO,MAXIMILIANO ALONSO LPEZ ARANCIBIA,2016-04-16,TIARE.ARANCIBIA.VERGARA@GMAIL.,NO INFORMADO,0
1,2611247,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17589680,5,17589680,5,FEMENINO,TITULAR MASCULINO,YENIFER VALERIA ROJAS ROMAN,1990-06-19,yenifer.rojas.r@gmail.com,SANTIAGO,954068927
2,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,17308982,1,FEMENINO,TITULAR MASCULINO,VALESKA BELEN SILVA BUSTOS,1989-08-04,kekita.bonita@hotmail.com,SANTIAGO,953810699
3,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,27712391,6,MASCULINO,HIJO,SANTIAGO JESS SALAZAR SILVA,2022-02-02,NO INFORMADO,NO INFORMADO,0
4,2611259,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,9711692,K,9711692,K,FEMENINO,TITULAR MASCULINO,ANA MARITZA CORTES LIZARDI,1964-08-04,ARIVERA.Y@ICLOUD.COM,FRANCISCO ALDAY 3457 COMPAÑIA ALTA- CIUDAD DEL...,962801697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,2612526,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16658639,9,27443865,7,FEMENINO,HIJA,MARA DOMINGA CORTS BRAVO,2021-01-05,NO INFORMADO,NO INFORMADO,0
2382,2612526,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16658639,9,16359223,1,FEMENINO,ESPOSA,CONSTANZA ALEJANDRA BRAVO NICOLAS,1986-09-24,NO INFORMADO,NO INFORMADO,984717424
2383,2612526,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16658639,9,23202250,7,MASCULINO,HIJO,VICENTE TOMS CORTS BRAVO,2009-12-17,NO INFORMADO,NO INFORMADO,0
2384,2612528,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,8867089,2,8867089,2,FEMENINO,TITULAR MASCULINO,MARIA SOLEDAD ESQUIROL PINILLA,1965-09-11,SOLESQUI@GMAIL.COM,PJE PERUANO SN - QUELLON - CHILOE,988388180


In [10]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)


' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [11]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23008\2221294489.py:70: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [12]:
archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2611266,2023-06-16,2024-06-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16647977,0,25353688,8,...,,166479770,253536888,2611266,Hombre,,,,Maximiliano Alonso,Lpez Arancibia
1,2611247,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17589680,5,17589680,5,...,954068927,175896805,175896805,2611247,Mujer,yenifer.rojas.r@gmail.com,Santiago,954068927,Yenifer Valeria,Rojas Roman
2,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,17308982,1,...,953810699,173089821,173089821,2611256,Mujer,kekita.bonita@hotmail.com,Santiago,953810699,Valeska Belen,Silva Bustos
3,2611256,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17308982,1,27712391,6,...,,173089821,277123916,2611256,Hombre,,,,Santiago Jess,Salazar Silva
4,2611259,2023-06-16,2024-06-15,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,9711692,K,9711692,K,...,962801697,9711692K,9711692K,2611259,Mujer,arivera.y@icloud.com,Francisco Alday 3457 Compañia Alta- Ciudad Del...,962801697,Ana Maritza,Cortes Lizardi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,2612526,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16658639,9,27443865,7,...,,166586399,274438657,2612526,Mujer,,,,Mara Dominga,Corts Bravo
2382,2612526,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16658639,9,16359223,1,...,984717424,166586399,163592231,2612526,Mujer,,,984717424,Constanza Alejandra,Bravo Nicolas
2383,2612526,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16658639,9,23202250,7,...,,166586399,232022507,2612526,Hombre,,,,Vicente Toms,Corts Bravo
2384,2612528,2023-06-27,2024-06-26,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,8867089,2,8867089,2,...,988388180,88670892,88670892,2612528,Mujer,solesqui@gmail.com,Pje Peruano Sn - Quellon - Chiloe,988388180,Maria Soledad,Esquirol Pinilla


In [13]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)


In [14]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value


""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """

' def revisaDict(ver):\n    numbers = ver\n    for key, value in numbers.items():\n        print(key, "->", value)\n        if key == "IdentificationId__pc":\n            return value '

In [15]:
results = sf.query_all(
    """
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Canal Individual'
   """
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["rutes"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["rutes"] = pacienteProgr["rutes"].astype(str).str.upper()

pacienteProgr = pacienteProgr[
    [
        "rutes",
    ]
]

""" Pacientes_SF.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
) """
pacienteProgr.shape

attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtphAAC')])
IdentificationId__pc -> 196364803
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpiAAC')])
IdentificationId__pc -> 154512675
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpjAAC')])
IdentificationId__pc -> 243180856
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpkAAC')])
IdentificationId__pc -> 14067401K
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtplAAC')])
IdentificationId__pc -> 268701028
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtsvAAC')])
IdentificationId__pc -> 101272702
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/A

(1206, 1)

In [16]:
pacienteProgr["rutes"] = pacienteProgr["rutes"].astype(str).str.upper()
pacienteProgr["rutes"] = pacienteProgr["rutes"].str.replace(" ", "")
pacienteProgr["rutes"] = pacienteProgr["rutes"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(pacienteProgr["rutes"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]

archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
)
archivoBCI.drop_duplicates(["Rut Carga", "Rut Titular"], keep="last")

archivoBCI = archivoBCI[
    ["Rut Carga", "sexo", "direccion", "email", "celular", "nombres", "apellidos"]
]

archivoBCI = archivoBCI.rename(
    columns={
        "Rut Carga": "IdentificationId__pc",
        "sexo": "HealthCloudGA__Gender__pc",
        "direccion": "BillingStreet",
        "email": "PersonEmail",
        "celular": "Phone",
        "nombres": "FirstName",
        "apellidos": "LastName",
    }
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(
    archivoBCI["PersonEmail"] != "", "True", "False"
)
archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
)


archivoBCI.columns

Index(['IdentificationId__pc', 'HealthCloudGA__Gender__pc', 'BillingStreet',
       'PersonEmail', 'Phone', 'FirstName', 'LastName', 'BillingCountry',
       'Identification_Type__c', 'Pa_s_del_Paciente__c', 'RecordTypeId',
       'Cuenta_validada__c', 'Validaci_n__c', 'Email_Seteado__c'],
      dtype='object')

In [93]:
""" derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)
 """
soql_query = """ select Id, Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Canal Individual' """
soql_result = sf.query_all(soql_query)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
df = df.drop(columns="Id")
list_of_records = df.to_dict("index")

""" derty = sf.quick_search("153298939")
print(derty) """

list_of_records[IdentificationId__pc]

{0: {'Paciente__r': OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
                              '/services/data/v42.0/sobjects/Account/0018c00002mVtphAAC')])),
               ('IdentificationId__pc', '196364803')])},
 1: {'Paciente__r': OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
                              '/services/data/v42.0/sobjects/Account/0018c00002mVtpiAAC')])),
               ('IdentificationId__pc', '154512675')])},
 2: {'Paciente__r': OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
                              '/services/data/v42.0/sobjects/Account/0018c00002mVtpjAAC')])),
               ('IdentificationId__pc', '243180856')])},
 3: {'Paciente__r': OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
       

In [101]:
list_of_records[0]["Paciente__r"]["IdentificationId__pc"]


'196364803'

In [118]:
derty["searchRecords"]

[OrderedDict([('attributes',
               OrderedDict([('type', 'Account'),
                            ('url',
                             '/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')])),
              ('Id', '0018c00002anTmeAAE')]),
 OrderedDict([('attributes',
               OrderedDict([('type', 'User'),
                            ('url',
                             '/services/data/v42.0/sobjects/User/0058c00000DSxZmAAL')])),
              ('Id', '0058c00000DSxZmAAL')]),
 OrderedDict([('attributes',
               OrderedDict([('type', 'Contact'),
                            ('url',
                             '/services/data/v42.0/sobjects/Contact/0038c000036iZ1TAAU')])),
              ('Id', '0038c000036iZ1TAAU')]),
 OrderedDict([('attributes',
               OrderedDict([('type', 'Task'),
                            ('url',
                             '/services/data/v42.0/sobjects/Task/00T8c00004uc71MEAQ')])),
              ('Id', '00T8c00004uc71MEAQ')]),


In [217]:
#derty = dict(sf.quick_search("MEOA010908MDFNRNA9"))
#prueba = "153298939"
derty = dict(sf.search("FIND {:prueba} RETURNING Account(IdentificationId__pc,Id)",prueba = 153298939))
derty


TypeError: search() got an unexpected keyword argument 'prueba'

In [201]:
print ( derty["searchRecords"][0]["Id"] + " " + derty["searchRecords"][0]["IdentificationId__pc"] )

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
derty = dict(sf.search("FIND {153298939}"))
if (derty["searchRecords"][0]["attributes"]['type'] == 'Account'):
    jote = derty["searchRecords"][0]["Id"]
    print (jote)
else:
    print("no")                      

In [ ]:
[OrderedDict(
            [
                ('attributes',
               OrderedDict(
                   [
                       ('type', 'Account'),
                        ('url','/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')
                    ]
                           )
                ),
              ('Id', '0018c00002anTmeAAE')
              ]
             ),